In [1]:
import pickle

from fft import *
from data_loading import *


In [52]:
n = 100000000
longitudes = np.random.randint(0, 576, size=n, dtype="uint16")
latitudes = np.random.randint(0, 361, size=n, dtype="uint16")
times = np.random.randint(0, 365 * 8, size=n, dtype="uint16")
levels = np.random.randint(0, 36, size=n, dtype="uint8")

east_wind_est = np.zeros(shape=n, dtype="float16")
north_wind_est = np.zeros(shape=n, dtype="float16")

east_wind = np.zeros(shape=n, dtype="float16")
north_wind = np.zeros(shape=n, dtype="float16")

data = pd.DataFrame({"lat": latitudes, "lon": longitudes, "lev": levels, "time": times,
                     "u_est": east_wind_est, "v_est": north_wind_est,
                     "u": east_wind, "v": north_wind})
data = data.sort_values(by="time", ignore_index=True)
data = data.set_index("time")
data.head()

,lat,lon,lev,u_est,v_est,u,v
time,,,,,,,
0,321,371,16,0.0,0.0,0.0,0.0
0,41,82,24,0.0,0.0,0.0,0.0
0,120,83,6,0.0,0.0,0.0,0.0
0,28,5,1,0.0,0.0,0.0,0.0
0,88,333,6,0.0,0.0,0.0,0.0


In [45]:
print(f"Size: {data.memory_usage().sum() / (1000 ** 2)} MB")
print(f"Duplicates: {100 * data.duplicated(subset=['lat', 'lon', 'lev', 'time']).sum() / n:.3f}%")


Size: 1500.0 MB
Duplicates: 0.228901%


In [3]:
estimate_quantile = 0.9935

In [55]:
i = 0
for mm in range(1, 13):
    for dd in range(1, monthrange(2001, mm)[1] + 1):
        for t in range(8):
            with open(f"models/3D-dft-daily/U/{estimate_quantile}/YAVG{mm:0>2}{dd:0>2}-{t * 3 + 1:0>2}:30.bin") as file:
                fft = pickle.load(file)
            subdata = data.loc[i]
            i += 1


,lat,lon,lev,u_est,v_est,u,v
time,,,,,,,
2,162,134,29,0.0,0.0,0.0,0.0
2,117,407,23,0.0,0.0,0.0,0.0
2,358,547,12,0.0,0.0,0.0,0.0
2,43,83,25,0.0,0.0,0.0,0.0
2,334,541,22,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2,234,172,22,0.0,0.0,0.0,0.0
2,338,14,31,0.0,0.0,0.0,0.0
2,280,147,27,0.0,0.0,0.0,0.0


In [4]:
with open(f"models/3D-dft-daily/U/{estimate_quantile}/YAVG0101-01:30.bin", "rb") as file:
    fft_raw = pickle.load(file)
    fft_real, fft_imag, fft_i_indices, fft_j_indices, fft_k_indices = fft_raw

fft_real = decode_zlib(fft_real, dtype="float16")
fft_imag = decode_zlib(fft_imag, dtype="float16")
fft = fft_real.astype("complex64") * 131072 + fft_imag.astype("complex64") * 131072j

fft_i_indices = decode_zlib(fft_i_indices)
fft_i_indices = decode_difference_uint8(fft_i_indices)

fft_j_indices = decode_zlib(fft_j_indices)
fft_j_indices = decode_difference_uint8(fft_j_indices)

fft_k_indices = decode_zlib(fft_k_indices)
fft_k_indices = decode_difference_uint8(fft_k_indices)

fft


array([ 4.1648128e+07+0.000000e+00j,  8.0192000e+04+3.186688e+06j,
       -3.0146560e+07-6.926336e+06j, ..., -1.1688000e+04+6.728000e+03j,
        6.7520000e+03-1.267200e+04j,  3.6220000e+03+1.270400e+04j])

In [5]:
actual = idft3_at_time(*fft_raw)

In [96]:
phi = 17
lam = 35
z = 15
actual[z, phi, lam]

-0.7811151397992238

In [65]:
import numpy.fft._pocketfft_internal as pfi

ifft = np.zeros((36, 361, 289), dtype="complex64")
for idx in range(len(fft)):
    ifft[fft_i_indices[idx], fft_j_indices[idx], fft_k_indices[idx]] = fft[idx]

In [99]:
def irfft3_at_level(a: np.ndarray, lev: int):
    out = np.swapaxes(a, axis1=0, axis2=-1)
    out = pfi.execute(out, False, False, 1 / 36)  # a, is_real, is_forward, 1 / norm
    out = np.moveaxis(out, -1, 0)

    out = out[lev, :, :]

    out = pfi.execute(out, False, False, 1 / 361)  # a, is_real, is_forward, 1 / norm

    return np.fft.irfft(out.T, 576, axis=-1, norm=None)


irfft3_at_level(ifft, lev=z)[phi, lam]

-0.7811151397992238

In [36]:
output = 0

for i in range(len(fft)):
    w = fft_i_indices[i]
    u = fft_j_indices[i]
    v = fft_k_indices[i]

    output += fft[i] * np.exp(2j * np.pi * ((u * phi / 361) + (v * lam / 576) + (w * z / 36)))

output /= (361 * 576 * 36)
output


(1.0392517891372426-1.5809750783778156j)